In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [ ]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style2/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/to_pggan/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '__'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style2/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=100, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%900 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:0.9536 | MainLoss:0.9310 | Alpha:0.0275 | SPLoss:0.8322 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6927 | MainLoss:0.6927 | SPLoss:0.9268 | CLSLoss:0.0000 | AUROC:0.5754
Test | 31/16 | Loss:0.7149 | MainLoss:0.7149 | SPLoss:0.9268 | CLSLoss:0.0000 | AUROC:0.0225

Epoch: [2 | 1000] LR: 0.103000
Train | 16/16 | Loss:0.7200 | MainLoss:0.6956 | Alpha:0.0274 | SPLoss:0.8872 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6927 | MainLoss:0.6927 | SPLoss:0.8524 | CLSLoss:0.0000 | AUROC:0.5588
Test | 31/16 | Loss:0.7051 | MainLoss:0.7051 | SPLoss:0.8524 | CLSLoss:0.0000 | AUROC:0.1297

Epoch: [3 | 1000] LR: 0.106000
Train | 16/16 | Loss:0.7168 | MainLoss:0.6952 | Alpha:0.0265 | SPLoss:0.8195 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6933 | MainLoss:0.6933 | SPLoss:0.7790 | CLSLoss:0.0000 | AUROC:0.5641
Test | 31/16 | Loss:0.7048 | MainLoss:0.7048 | SPLoss:0.7790 | CLSLoss:0.0000 | AUROC:0.2240

Epoch: [4 | 100

Train | 16/16 | Loss:0.9971 | MainLoss:0.4173 | Alpha:0.0276 | SPLoss:20.9926 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1942 | MainLoss:0.1942 | SPLoss:19.1745 | CLSLoss:0.0000 | AUROC:0.9991
Test | 31/16 | Loss:0.5478 | MainLoss:0.5478 | SPLoss:19.1746 | CLSLoss:0.0000 | AUROC:0.9952

Epoch: [27 | 1000] LR: 0.178000
Train | 16/16 | Loss:0.8953 | MainLoss:0.4304 | Alpha:0.0260 | SPLoss:17.8656 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.3074 | MainLoss:0.3074 | SPLoss:16.5499 | CLSLoss:0.0000 | AUROC:0.9981
Test | 31/16 | Loss:0.5628 | MainLoss:0.5628 | SPLoss:16.5499 | CLSLoss:0.0000 | AUROC:0.8461

Epoch: [28 | 1000] LR: 0.181000
Train | 16/16 | Loss:0.8414 | MainLoss:0.4360 | Alpha:0.0264 | SPLoss:15.3600 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2111 | MainLoss:0.2111 | SPLoss:14.1016 | CLSLoss:0.0000 | AUROC:0.9988
Test | 31/16 | Loss:0.5132 | MainLoss:0.5132 | SPLoss:14.1016 | CLSLoss:0.0000 | AUROC:0.9935

Epoch: [29 | 1000] LR: 0.184000
Trai

Train | 16/16 | Loss:0.7647 | MainLoss:0.4478 | Alpha:0.0256 | SPLoss:12.2905 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1306 | MainLoss:0.1306 | SPLoss:11.0187 | CLSLoss:0.0000 | AUROC:0.9994
Test | 31/16 | Loss:0.9202 | MainLoss:0.9202 | SPLoss:11.0187 | CLSLoss:0.0000 | AUROC:0.4247

Epoch: [52 | 1000] LR: 0.253000
Train | 16/16 | Loss:0.8021 | MainLoss:0.5159 | Alpha:0.0276 | SPLoss:10.3459 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1890 | MainLoss:0.1890 | SPLoss:9.3145 | CLSLoss:0.0000 | AUROC:0.9993
Test | 31/16 | Loss:0.8038 | MainLoss:0.8038 | SPLoss:9.3145 | CLSLoss:0.0000 | AUROC:0.4420

Epoch: [53 | 1000] LR: 0.256000
Train | 16/16 | Loss:0.6693 | MainLoss:0.4434 | Alpha:0.0270 | SPLoss:8.3797 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2265 | MainLoss:0.2265 | SPLoss:7.4109 | CLSLoss:0.0000 | AUROC:0.9994
Test | 31/16 | Loss:0.7533 | MainLoss:0.7533 | SPLoss:7.4109 | CLSLoss:0.0000 | AUROC:0.4434

Epoch: [54 | 1000] LR: 0.259000
Train | 1

Train | 16/16 | Loss:0.6198 | MainLoss:0.4818 | Alpha:0.0264 | SPLoss:5.2055 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1756 | MainLoss:0.1756 | SPLoss:4.7857 | CLSLoss:0.0000 | AUROC:0.9996
Test | 31/16 | Loss:0.8279 | MainLoss:0.8279 | SPLoss:4.7857 | CLSLoss:0.0000 | AUROC:0.3775

Epoch: [77 | 1000] LR: 0.328000
Train | 16/16 | Loss:0.6027 | MainLoss:0.4375 | Alpha:0.0278 | SPLoss:5.9840 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1704 | MainLoss:0.1704 | SPLoss:5.8313 | CLSLoss:0.0000 | AUROC:0.9997
Test | 31/16 | Loss:0.8393 | MainLoss:0.8393 | SPLoss:5.8313 | CLSLoss:0.0000 | AUROC:0.3804

Epoch: [78 | 1000] LR: 0.331000
Train | 16/16 | Loss:0.5903 | MainLoss:0.4465 | Alpha:0.0277 | SPLoss:5.1654 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2857 | MainLoss:0.2857 | SPLoss:4.6590 | CLSLoss:0.0000 | AUROC:0.9975
Test | 31/16 | Loss:0.7331 | MainLoss:0.7331 | SPLoss:4.6590 | CLSLoss:0.0000 | AUROC:0.4188

Epoch: [79 | 1000] LR: 0.334000
Train | 16/16

Test | 31/16 | Loss:0.7723 | MainLoss:0.7723 | SPLoss:2715.7441 | CLSLoss:0.0000 | AUROC:0.5714

Epoch: [101 | 1000] LR: 0.400000
Train | 16/16 | Loss:61.3824 | MainLoss:0.4740 | Alpha:0.0266 | SPLoss:2272.1147 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2336 | MainLoss:0.2336 | SPLoss:1854.6195 | CLSLoss:0.0000 | AUROC:0.9982
Test | 31/16 | Loss:0.7633 | MainLoss:0.7633 | SPLoss:1854.6163 | CLSLoss:0.0000 | AUROC:0.2179

Epoch: [102 | 1000] LR: 0.400000
Train | 16/16 | Loss:41.8196 | MainLoss:0.4990 | Alpha:0.0264 | SPLoss:1552.3323 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2509 | MainLoss:0.2509 | SPLoss:1272.0585 | CLSLoss:0.0000 | AUROC:0.9954
Test | 31/16 | Loss:0.7575 | MainLoss:0.7575 | SPLoss:1272.0582 | CLSLoss:0.0000 | AUROC:0.4528

Epoch: [103 | 1000] LR: 0.399999
Train | 16/16 | Loss:28.6358 | MainLoss:0.4477 | Alpha:0.0262 | SPLoss:1065.3915 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1447 | MainLoss:0.1447 | SPLoss:873.2485 | CLSLoss:0.0

...

Train | 16/16 | Loss:1.5914 | MainLoss:0.3528 | Alpha:0.0267 | SPLoss:46.4058 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2323 | MainLoss:0.2323 | SPLoss:45.6948 | CLSLoss:0.0000 | AUROC:0.9985
Test | 31/16 | Loss:0.4053 | MainLoss:0.4053 | SPLoss:45.6948 | CLSLoss:0.0000 | AUROC:0.9825

Epoch: [397 | 1000] LR: 0.032008
Train | 16/16 | Loss:1.5995 | MainLoss:0.3572 | Alpha:0.0276 | SPLoss:45.0233 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2250 | MainLoss:0.2250 | SPLoss:44.3520 | CLSLoss:0.0000 | AUROC:0.9986
Test | 31/16 | Loss:0.4277 | MainLoss:0.4277 | SPLoss:44.3520 | CLSLoss:0.0000 | AUROC:0.9790

Epoch: [398 | 1000] LR: 0.031958
Train | 16/16 | Loss:1.5441 | MainLoss:0.3561 | Alpha:0.0271 | SPLoss:43.7882 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1688 | MainLoss:0.1688 | SPLoss:43.0748 | CLSLoss:0.0000 | AUROC:0.9987
Test | 31/16 | Loss:0.5023 | MainLoss:0.5023 | SPLoss:43.0748 | CLSLoss:0.0000 | AUROC:0.9781

Epoch: [399 | 1000] LR: 0.031908
T

Test | 31/16 | Loss:0.4887 | MainLoss:0.4887 | SPLoss:38.4722 | CLSLoss:0.0000 | AUROC:0.9775

Epoch: [421 | 1000] LR: 0.003077
Train | 16/16 | Loss:1.3665 | MainLoss:0.3377 | Alpha:0.0268 | SPLoss:38.4178 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1811 | MainLoss:0.1811 | SPLoss:38.3647 | CLSLoss:0.0000 | AUROC:0.9985
Test | 31/16 | Loss:0.4919 | MainLoss:0.4919 | SPLoss:38.3647 | CLSLoss:0.0000 | AUROC:0.9770

Epoch: [422 | 1000] LR: 0.003072
Train | 16/16 | Loss:1.3319 | MainLoss:0.3429 | Alpha:0.0258 | SPLoss:38.3179 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1830 | MainLoss:0.1830 | SPLoss:38.2618 | CLSLoss:0.0000 | AUROC:0.9985
Test | 31/16 | Loss:0.4877 | MainLoss:0.4877 | SPLoss:38.2618 | CLSLoss:0.0000 | AUROC:0.9771

Epoch: [423 | 1000] LR: 0.003066
Train | 16/16 | Loss:1.3083 | MainLoss:0.3503 | Alpha:0.0251 | SPLoss:38.2151 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1825 | MainLoss:0.1825 | SPLoss:38.1647 | CLSLoss:0.0000 | AUROC:0.9985
T

Test | 128/16 | Loss:0.1820 | MainLoss:0.1820 | SPLoss:35.9563 | CLSLoss:0.0000 | AUROC:0.9983
Test | 31/16 | Loss:0.4878 | MainLoss:0.4878 | SPLoss:35.9564 | CLSLoss:0.0000 | AUROC:0.9787

Epoch: [446 | 1000] LR: 0.002941
Train | 16/16 | Loss:1.3023 | MainLoss:0.3341 | Alpha:0.0270 | SPLoss:35.9129 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1908 | MainLoss:0.1908 | SPLoss:35.8603 | CLSLoss:0.0000 | AUROC:0.9982
Test | 31/16 | Loss:0.4738 | MainLoss:0.4738 | SPLoss:35.8603 | CLSLoss:0.0000 | AUROC:0.9786

Epoch: [447 | 1000] LR: 0.002936
Train | 16/16 | Loss:1.2852 | MainLoss:0.3310 | Alpha:0.0266 | SPLoss:35.8168 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1793 | MainLoss:0.1793 | SPLoss:35.7673 | CLSLoss:0.0000 | AUROC:0.9982
Test | 31/16 | Loss:0.4979 | MainLoss:0.4979 | SPLoss:35.7673 | CLSLoss:0.0000 | AUROC:0.9780

Epoch: [448 | 1000] LR: 0.002930
Train | 16/16 | Loss:1.3035 | MainLoss:0.3343 | Alpha:0.0271 | SPLoss:35.7184 | CLSLoss:0.0000 | AUROC:0.0000
T

Train | 16/16 | Loss:1.2520 | MainLoss:0.3428 | Alpha:0.0269 | SPLoss:33.7392 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1833 | MainLoss:0.1833 | SPLoss:33.6952 | CLSLoss:0.0000 | AUROC:0.9984
Test | 31/16 | Loss:0.4879 | MainLoss:0.4879 | SPLoss:33.6951 | CLSLoss:0.0000 | AUROC:0.9774

Epoch: [471 | 1000] LR: 0.002800
Train | 16/16 | Loss:1.2756 | MainLoss:0.3417 | Alpha:0.0277 | SPLoss:33.6579 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1841 | MainLoss:0.1841 | SPLoss:33.6092 | CLSLoss:0.0000 | AUROC:0.9983
Test | 31/16 | Loss:0.4910 | MainLoss:0.4910 | SPLoss:33.6091 | CLSLoss:0.0000 | AUROC:0.9779

Epoch: [472 | 1000] LR: 0.002794
Train | 16/16 | Loss:1.2221 | MainLoss:0.3294 | Alpha:0.0266 | SPLoss:33.5712 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1871 | MainLoss:0.1871 | SPLoss:33.5249 | CLSLoss:0.0000 | AUROC:0.9984
Test | 31/16 | Loss:0.4823 | MainLoss:0.4823 | SPLoss:33.5249 | CLSLoss:0.0000 | AUROC:0.9779

Epoch: [473 | 1000] LR: 0.002789
T

Test | 31/16 | Loss:0.4994 | MainLoss:0.4994 | SPLoss:31.7914 | CLSLoss:0.0000 | AUROC:0.9746

Epoch: [495 | 1000] LR: 0.002660
Train | 16/16 | Loss:1.2028 | MainLoss:0.3390 | Alpha:0.0272 | SPLoss:31.7537 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1708 | MainLoss:0.1708 | SPLoss:31.7150 | CLSLoss:0.0000 | AUROC:0.9984
Test | 31/16 | Loss:0.5130 | MainLoss:0.5130 | SPLoss:31.7151 | CLSLoss:0.0000 | AUROC:0.9760

Epoch: [496 | 1000] LR: 0.002654
Train | 16/16 | Loss:1.2185 | MainLoss:0.3286 | Alpha:0.0281 | SPLoss:31.6791 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1821 | MainLoss:0.1821 | SPLoss:31.6360 | CLSLoss:0.0000 | AUROC:0.9984
Test | 31/16 | Loss:0.4908 | MainLoss:0.4908 | SPLoss:31.6360 | CLSLoss:0.0000 | AUROC:0.9763

Epoch: [497 | 1000] LR: 0.002648
Train | 16/16 | Loss:1.1938 | MainLoss:0.3292 | Alpha:0.0274 | SPLoss:31.6027 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.1831 | MainLoss:0.1831 | SPLoss:31.5613 | CLSLoss:0.0000 | AUROC:0.9984
T